In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('2021-01-07.csv',parse_dates=['Date'],index_col='Date')
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)

In [3]:
trading_days=250                  #Trading days per year (automated)

In [4]:
returnsh=df.pct_change()                  #Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)           #calculating daily returns of the stocks in the portfolio

In [5]:
returnsh

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.004116,-0.017328,-0.003911,-0.003210,0.009825,0.024238,-0.013265,-0.004311,0.034958,-0.013568,...,-0.002317,0.043351,-0.026372,-0.013417,-0.007547,0.004776,-0.006674,0.001050,0.020061,0.000246
2,-0.004670,0.010079,0.022950,0.003701,-0.019795,0.009395,-0.015999,-0.003027,0.053465,-0.020043,...,-0.020953,0.004588,-0.001619,-0.006128,-0.006777,0.057034,-0.002407,-0.019236,-0.009251,0.000861


In [6]:
returnso=returnsh.copy()                  #this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
    for i in range(0,len(df)):
      if returnso[cols][i] > 0:
        returnso[cols][i]=0               #Here, returnso would mean return considered for sortino ratio

In [7]:
returnso

,ASIANPAINT,BAJFINANCE,BAJAJFINSV,BRITANNIA,DIVISLAB,GRASIM,HCLTECH,HDFCBANK,HINDALCO,HINDUNILVR,...,NESTLEIND,POWERGRID,RELIANCE,TCS,TATACONSUM,TATASTEEL,TECHM,TITAN,ULTRACEMCO,WIPRO
0,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1,0.00000,-0.017328,-0.003911,-0.00321,0.000000,0.0,-0.013265,-0.004311,0.0,-0.013568,...,-0.002317,0.0,-0.026372,-0.013417,-0.007547,0.0,-0.006674,0.000000,0.000000,0.0
2,-0.00467,0.000000,0.000000,0.00000,-0.019795,0.0,-0.015999,-0.003027,0.0,-0.020043,...,-0.020953,0.0,-0.001619,-0.006128,-0.006777,0.0,-0.002407,-0.019236,-0.009251,0.0


In [8]:
covmatsh=returnsh.cov()*trading_days      #Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso=returnso.cov()*trading_days      #Annualised covariance matrix calculated wrt returnso i.e. used to calculate sortino ratio

In [9]:
num_portfolios = 50000                   #initializing number of portfolios to 50000; referred from Wang et al (2020) (science direct)
num_assets = len(df.columns)              #initializing number of stocks/assets considered in the portfolio
risk_free_rate = 0.0358                   #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [10]:
#2021_chen etal_Mean–variance portfolio optimization using machine learning-based stock price prediction
#Repeat the process 50,000times. From a statistical point of view, 50,000 random portfolios cover most possible portfolios with different weights and aresufficiently representative

In [11]:
portfolio_returns = []                    #initializing an empty list for portfolio returns
portfolio_volatility =[]                  #initializing an empty list for portfolio risk
stock_weights =[]                         #initializing an empty list for portfolio weights
semi_deviation =[]                        #initializing an empty list for portfolio semi-deviation
sharpe =[]                                #initializing an empty list for portfolio sharpe ratio
sortino =[]                               #initializing an empty list for portfolio sortino ratio

In [12]:
def ratio(a,b,c):                         #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  return (a-c)/b                          #a => annual return, c => risk_free_rate, b => deviation (standard for sharpe, semi for sortino)

In [ ]:
for single_portfolio in range(num_portfolios):                  #iterating forloop for 50000 times to generate 50000 portfolios
  weights = np.random.random(num_assets)                        #initializing random weights
  weights /= np.sum(weights)                                    #No Short Selling Allowed => weights add up to 1   "x = x+y" => "x+=y"    weights = weights/np.sum(weights)  
  returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
  varsh=np.dot(weights.T,np.dot(covmatsh,weights))              #calculating portfolio varience wrt calculating sharpe ratio
  varso=np.dot(weights.T,np.dot(covmatso,weights))              #calculating portfolio varience wrt calculating sortino ratio
  volatility_temp = np.sqrt(varsh)                              #portfolio risk
  semi_temp = np.sqrt(varso)                                    #portfolio semi-deviation
  shtemp = ratio(returns_temp,volatility_temp,risk_free_rate)   #calculating sharpe ratio
  sotemp = ratio(returns_temp,semi_temp,risk_free_rate)         #calculating sortino ratio
  portfolio_returns.append(returns_temp)                       
  portfolio_volatility.append(volatility_temp)
  stock_weights.append(weights)
  sharpe.append(shtemp)
  sortino.append(sotemp)
  semi_deviation.append(semi_temp)

In [ ]:
portfolio = {'Returns' : portfolio_returns, 'Standard Deviation' : portfolio_volatility, 'Semi-Deviation' : semi_deviation, 'Sharpe Ratio' : sharpe, 
             'Sortino Ratio' : sortino}    
#here, 'portfolio' is a dictionary which will be used to create dataframe where each row will be a portfolio

In [ ]:
for counter,symbol in enumerate(df.columns):
  portfolio[symbol + " Weight"] = [Weight[counter] for Weight in stock_weights] 
#to the dictionary (named 'portfolio'), weights for each symbol are added in so as to be displayed in the dataframe

In [ ]:
pc = pd.DataFrame(portfolio)         #making the final dataframe where data of 50000 portfolios is appended (subject to be saved, whose code is to be written)

In [ ]:
pc=pc*100                                       #Converting everything to percentage
pc['Sharpe Ratio']=pc['Sharpe Ratio']/100       #leaving ratios as it is
pc['Sortino Ratio']=pc['Sortino Ratio']/100

In [ ]:
#pc.to_csv('portfolios_by_MV.csv')  #saving the portfolios data

In [ ]:
max_sharpe=pc['Sharpe Ratio'].max()                                             #Best optimised portfolio wrt sharpe ratio
max_sharpe_portfolio=pc.loc[pc['Sharpe Ratio'] == max_sharpe]
max_sharpe_portfolio

In [ ]:
max_sortino=pc['Sortino Ratio'].max()                                            #Best optimised portfolio wrt sortino ratio
max_sortino_portfolio=pc.loc[pc['Sortino Ratio'] == max_sortino]
max_sortino_portfolio

In [ ]:
#code for visualization is to be written

In [ ]:
pc_sharpe=pc.drop(columns=['Sortino Ratio','Semi-Deviation'])

In [ ]:
pc_sharpe_top10=pc_sharpe.sort_values(by=['Sharpe Ratio'],ascending=False).head(10)

In [ ]:
pc_sharpe_top10.to_csv('Sharpe_Top10_MV.csv')

In [ ]:
pc_sharpe_bottom10=pc_sharpe.sort_values(by=['Sharpe Ratio'],ascending=False).tail(10)

In [ ]:
pc_sharpe_bottom10.to_csv('Sharpe_Bottom10_MV.csv')

In [ ]:
pc_sharpe_bottom10


In [ ]:
pc_sharpe_top10

In [ ]:
sharpe_optimal_portfolio=pc_sharpe_top10.head(1)

In [ ]:
sharpe_optimal_portfolio.to_csv('Sharpe_Optimal_MV.csv')

In [ ]:
sharpe_optimal_portfolio.T

In [ ]:
pc_sortino=pc.drop(columns=['Sharpe Ratio','Standard Deviation'])

In [ ]:
pc_sortino_top10=pc_sortino.sort_values(by=['Sortino Ratio'],ascending=False).head(10)

In [ ]:
pc_sortino_top10.to_csv('Sortino_Top10_MV.csv')

In [ ]:
pc_sortino_top10

In [ ]:
pc_sortino_bottom10=pc_sortino.sort_values(by=['Sortino Ratio'],ascending=False).tail(10)

In [ ]:
pc_sortino_bottom10.to_csv('Sortino_Bottom10_MV.csv')

In [ ]:
pc_sortino_bottom10

In [ ]:
sortino_optimal_portfolio=pc_sortino_top10.head(1)

In [ ]:
sortino_optimal_portfolio.to_csv('Sortino_Optimal_MV.csv')

In [ ]:
sortino_optimal_portfolio.T

In [ ]:
edf=pd.read_csv('n50.csv',parse_dates=['Date'],index_col='Date')
edf = edf.loc["2016-01-01" : ]   
edf.reset_index(drop=True, inplace=True)
(edf / edf.iloc[0]*100).plot(figsize=(10,5))


In [ ]:
plt.style.use('seaborn')
pc_sharpe.plot.scatter(x='Standard Deviation', y='Returns', marker='o',figsize=(20, 10), grid=True)
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

In [ ]:
#GREEN STAR-> Optimal Risky Portfolio
#RED STAR-> Minimum Volatility
plt.subplots(figsize=(20, 10))
plt.xlabel("Expected Volatility")
plt.ylabel("Returns")
plt.scatter(pc_sharpe['Standard Deviation'], pc_sharpe['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(minimum_risk_portfolio['Standard Deviation'], minimum_risk_portfolio['Returns'], color='r', marker='*', s=500)
plt.scatter(sharpe_optimal_portfolio['Standard Deviation'], sharpe_optimal_portfolio['Returns'], color='g', marker='*', s=500)
